In [2]:
import pandas as pd
import seaborn as sns
import sklearn.metrics
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import xgboost as xgb

In [3]:
le = preprocessing.LabelEncoder()

In [4]:
df = pd.read_csv('../data/data.csv')

In [16]:
df

,ds_search_11/20/18,ds_search_11/21/18,ds_search_11/22/18,ds_search_11/23/18,ds_search_11/25/18,ds_search_11/26/18,ds_book_11/20/18,ds_book_11/21/18,ds_book_11/22/18,ds_book_11/23/18,...,listing_num_recent_reservations,listing_monthly_discount,listing_weekly_discount,False,True,MIN,25%,AVG,75%,MAX
0,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,1,1,0,1,0,1,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177583,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
177584,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
177585,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
177586,0,0,0,0,0,1,0,0,0,0,...,0,1,1,0,1,0,0,1,0,0


In [19]:
df

,index,ds_search_11/20/18,ds_search_11/21/18,ds_search_11/22/18,ds_search_11/23/18,ds_search_11/25/18,ds_search_11/26/18,ds_book_11/20/18,ds_book_11/21/18,ds_book_11/22/18,...,listing_num_recent_reservations,listing_monthly_discount,listing_weekly_discount,False,True,MIN,25%,AVG,75%,MAX
0,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,1,0,0,1,0,0
1,1,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,0
2,2,0,0,0,0,0,1,0,0,0,...,0,1,1,0,1,0,1,0,0,0
3,3,0,0,0,0,0,1,0,0,0,...,0,1,1,1,0,0,0,0,0,0
4,4,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177583,177583,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,1,0,0
177584,177584,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
177585,177585,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,1,0,0
177586,177586,0,0,0,0,0,1,0,0,0,...,0,1,1,0,1,0,0,1,0,0


In [5]:
X = df.drop(columns = 'label')
X = X.apply(le.fit_transform)
y = df['label']
y = y.values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, random_state=42)

In [6]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test) 

In [7]:
model = xgb.XGBClassifier(  
    #tree_method='gpu_hist',
    booster='gbtree',
    objective='rank:pairwise',
    random_state=42, 
    learning_rate=0.01,
    colsample_bytree=0.9, 
    eta=0.05, 
    max_depth=6, 
    n_estimators=110, 
    subsample=0.75 
    )

model.fit(X_train, y_train)

/Users/jazmiahenry/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/jazmiahenry/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[15:04:28] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9,
              enable_categorical=False, eta=0.05, gamma=0, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=110, n_jobs=10, num_parallel_tree=1,
              objective='multi:softprob', predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=None, subsample=0.75,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [8]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [9]:
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

In [13]:
mse

0.9329459197693538

In [10]:
rmse

0.9658912566999216